# 1. Input

### 1. 경로설정

우선 os, sys 라이브러리를 가져와 상대경로의 기준을 설정합니다.

#### 이 과정은 사용자가 어떤 디렉토리에서 활동하더라도 상대 경로를 활용할 수 있도록 합니다.

In [1]:
import os
import sys

path_name = os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd())))
sys.path.append(path_name)
print(path_name)

/Users/moonygo/Documents/NextoptEngine_light


### 2. Read Libraries

In [2]:
from Engine.Data.IO import IO
from Engine.Data.transformer import Transformer
from Engine.Model.standard_model import *
from Engine.Model.standard_model import HoltWinters, AutoRegressiveNN, TBATS, AutoArima

In [3]:
import pandas as pd

### 3. Read Data

In [4]:
df = IO.read_pickle(path_name + '/Data/발주/order.pkl')

In [5]:
df.head()

,회사코드,일자,유형코드,수량
0,298141,20100101,1815,8
1,296285,20100101,1815,20
2,295537,20100101,1815,4
3,294932,20100101,1815,10
4,294837,20100101,1815,10


# 2. Preprocessing

### 1. Restruct Data by date & pallet

In [6]:
df_new = df.transformer.aggregate()

In [7]:
df_new.head()

,1025,1041,1057,1091,1111,1117,1119,1127,1163,1216,...,1797,1800,1815,1817,1852,1853,1878,1891,total,weekday
ds,,,,,,,,,,,,,,,,,,,,,
2010-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,2652.0,NaN,NaN,NaN,NaN,NaN,2662.0,4
2010-01-02,192.0,6260.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,26443.0,NaN,NaN,459.0,626.0,NaN,39352.0,5
2010-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,404.0,NaN,NaN,NaN,NaN,NaN,404.0,6
2010-01-04,NaN,5141.0,NaN,320.0,NaN,NaN,NaN,NaN,NaN,NaN,...,576.0,NaN,22924.0,NaN,NaN,3445.0,3305.0,NaN,44906.0,0
2010-01-05,130.0,5494.0,NaN,1186.0,NaN,NaN,NaN,NaN,NaN,16.0,...,294.0,NaN,39081.0,320.0,NaN,3861.0,4362.0,238.0,69481.0,1


### 2. Split Train/Test Data

In [8]:
df_train, df_test= df_new[['1815']][:-30], df_new[['1815']][-30:]

In [9]:
df_train.columns = ['y']
df_test.columns = ['y']

In [10]:
df_train.tail()

,y
ds,
2017-10-27,80121.0
2017-10-28,49909.0
2017-10-29,320.0
2017-10-30,74045.0
2017-10-31,79806.0


In [11]:
df_test.head()

,y
ds,
2017-11-01,75861.0
2017-11-02,83256.0
2017-11-03,82649.0
2017-11-04,42707.0
2017-11-05,105.0


# 3. Run Engine Models

### 1. Exponential Moving Average

In [12]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [13]:
model = ExponentialMovingAverage(train = df_train, test = df_test, frequency=365)

In [14]:
model.fit_and_forecast(horizon=30)

In [15]:
model.fit_df.head()

,yhat
ds,
2010-01-01,NaN
2010-01-02,2652.000000
2010-01-03,3160.922949
2010-01-04,3101.948490
2010-01-05,3525.970049


In [16]:
model.fit_df[-10:]

,yhat
ds,
2017-10-22,60977.055480
2017-10-23,59673.076943
2017-10-24,60014.097098
2017-10-25,60501.155953
2017-10-26,60908.936997
2017-10-27,61383.720609
2017-10-28,61784.537367
2017-10-29,61530.502921
2017-10-30,60221.124178


### 2. STL : presently not working (kernel dies)

model = STL(train = df_train, test = df_test, frequency=365)

model.fit_and_forecast(horizon=30)

model.fit_df.head()

model.forecast_df.head()

### 3. HoltWinters

In [17]:
model = HoltWinters(train = df_train, test = df_test, frequency=365)

In [18]:
model.fit_and_forecast(horizon=30)

In [19]:
model.fit_df.head()

,yhat,level,trend,season
ds,,,,
2010-01-01,NaN,NaN,NaN,NaN
2010-01-02,NaN,NaN,NaN,NaN
2010-01-03,NaN,NaN,NaN,NaN
2010-01-04,NaN,NaN,NaN,NaN
2010-01-05,NaN,NaN,NaN,NaN


In [20]:
model.forecast_df.head()

,yhat,yhat_lower_80,yhat_upper_80,yhat_lower_95,yhat_upper_95
ds,,,,,
2017-11-01,71255.175248,37882.615967,104627.734529,20216.250229,122294.100268
2017-11-02,74715.414854,41342.753221,108088.076487,23676.333301,125754.496408
2017-11-03,76839.743778,43466.977696,110212.509861,25800.502483,127878.985074
2017-11-04,76580.199029,43207.326378,109953.071681,25540.794751,127619.603308
2017-11-05,60747.360326,27374.378966,94120.341686,9707.789792,111786.930860


### 4. Arima with no seasonality

In [21]:
model = AutoArima(train=df_train, test=df_test, frequency=365, seasonal=False)

In [22]:
model.fit_and_forecast(horizon=30)

In [23]:
model.fit_df.head()

,yhat
ds,
2010-01-01,2649.358735
2010-01-02,10657.771251
2010-01-03,15407.840470
2010-01-04,1680.709126
2010-01-05,21756.251495


In [24]:
model.forecast_df.head()

,yhat,yhat_lower_80,yhat_upper_80,yhat_lower_95,yhat_upper_95
ds,,,,,
2017-11-01,64549.577719,40353.923264,88745.232173,27545.517493,101553.637944
2017-11-02,48601.160795,23021.244809,74181.076781,9480.055247,87722.266343
2017-11-03,65995.870888,39883.792577,92107.949198,26060.893298,105930.848477
2017-11-04,68463.673735,40547.109364,96380.238106,25768.972866,111158.374605
2017-11-05,56168.170291,28027.803585,84308.536997,13131.193287,99205.147294


### 5. Arima with seasonality

In [25]:
model = AutoArima(train=df_train, test=df_test, frequency=365, seasonal=True)

In [26]:
model.fit_and_forecast(horizon=30)

In [27]:
model.fit_df.head()

,yhat
ds,
2010-01-01,2649.358735
2010-01-02,10657.771251
2010-01-03,15407.840470
2010-01-04,1680.709126
2010-01-05,21756.251495


In [28]:
model.forecast_df.head()

,yhat,yhat_lower_80,yhat_upper_80,yhat_lower_95,yhat_upper_95
ds,,,,,
2017-11-01,64549.577719,40353.923264,88745.232173,27545.517493,101553.637944
2017-11-02,48601.160795,23021.244809,74181.076781,9480.055247,87722.266343
2017-11-03,65995.870888,39883.792577,92107.949198,26060.893298,105930.848477
2017-11-04,68463.673735,40547.109364,96380.238106,25768.972866,111158.374605
2017-11-05,56168.170291,28027.803585,84308.536997,13131.193287,99205.147294


이후에는 Multi-Seasonality를 적용할 수 있는 알고리즘입니다.

#### 이 때에는 frequency를 list로 작성해야 합니다.

### 6. TBATS with single seasonality

In [29]:
model = TBATS(train=df_train, test=df_test, frequency=[365])

In [30]:
model.fit_and_forecast(horizon=30)

In [31]:
model.fit_df.head()

,yhat
ds,
2010-01-01,27458.407373
2010-01-02,24528.691359
2010-01-03,36222.346247
2010-01-04,24823.328246
2010-01-05,41311.431246


In [32]:
model.forecast_df.head()

,yhat,yhat_lower_80,yhat_upper_80,yhat_lower_95,yhat_upper_95
ds,,,,,
2017-11-01,78575.807740,56967.097387,100184.518093,45528.137101,111623.478379
2017-11-02,81108.855057,58842.802417,103374.907697,47055.866145,115161.843969
2017-11-03,53559.762375,31236.115812,75883.408938,19418.691158,87700.833592
2017-11-04,45408.901662,22926.769111,67891.034212,11025.447044,79792.356280
2017-11-05,40126.925322,17627.542085,62626.308559,5717.088055,74536.762588


### 7. TBATS with multi seasonality

In [33]:
model = TBATS(train=df_train, test=df_test, frequency=[7,365])

In [34]:
model.fit_and_forecast(horizon=30)

In [35]:
model.fit_df.head()

,yhat
ds,
2010-01-01,42149.769491
2010-01-02,2599.139595
2010-01-03,-14842.402075
2010-01-04,41070.665161
2010-01-05,42342.978374


In [36]:
model.forecast_df.head()

,yhat,yhat_lower_80,yhat_upper_80,yhat_lower_95,yhat_upper_95
ds,,,,,
2017-11-01,76573.920931,63954.615186,89193.226675,57274.358248,95873.483613
2017-11-02,73290.335748,59304.403961,87276.267536,51900.698897,94679.972600
2017-11-03,72370.211289,57153.854247,87586.568331,49098.801398,95641.621179
2017-11-04,41006.195765,24598.565089,57413.826440,15912.890071,66099.501458
2017-11-05,521.589072,-16977.155568,18020.333713,-26240.431429,27283.609573


## --- Further Develop Plan ---

### 8. AutoRegressiveNN with single seasonality: presently not working (kernel dies)

model = AutoRegressiveNN(train=df_train, test=df_test, frequency=[365])

model.fit_and_forecast(horizon=30)

model.fit_df.head()

model.forecast_df.head()

### 9. AutoRegressiveNN with multi seasonality: presently not working (kernel dies)

model = AutoRegressiveNN(train=df_train, test=df_test, frequency=[7,365])

model.fit_and_forecast(horizon=30)

model.fit_df.head()

model.forecast_df.head()

# 4. Check the Evaluation Rate of Result

result= Evaluator(df_test, model.forecast_df)

result.rmse

result.mape

result.smape